In [4]:
# !pip install git+https://github.com/huggingface/transformers.git
!pip install transformers[tf-cpu]

     |████████████████████████████████| 2.3MB 8.6MB/s 
     |████████████████████████████████| 901kB 47.6MB/s 
     |████████████████████████████████| 3.3MB 42.7MB/s 
     |████████████████████████████████| 168.3MB 85kB/s 
     |████████████████████████████████| 81kB 13.9MB/s 
     |████████████████████████████████| 102kB 16.1MB/s 
     |████████████████████████████████| 12.2MB 42.2MB/s 
     |████████████████████████████████| 92kB 14.6MB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115943 sha256=6954eb6e026ea521ac373bb841c730e5dad9da02cd78af8808dbec4aa1a9f03f
  Stored in directory: /root/.cache/pip/wheels/af/19/30/1ea0cad502dcb4e66ed5a690279628c827aea38bbbab75d5ed
Successfully built fire


In [5]:
!python --version

Python 3.7.10


In [6]:
from transformers import DistilBertTokenizer
from transformers import TFDistilBertForSequenceClassification
import tensorflow as tf
import pandas as pd
import json
import gc

## Get data from this link:- (https://drive.google.com/file/d/19LM5l8xKkQ5L7yipKrTAaUAlocgd7vaE/view?usp=sharing)

In [7]:
from google.colab import files
uploaded = files.upload()

Saving train.csv to train.csv


In [8]:
col_names=['text','B','category']
df = pd.read_csv('train.csv',names=col_names,header=None)
df.head()

,text,B,category
0,BOS Sir I created a new env with python 3.7 bu...,O O O O O O O O O O O O O O O O O O O O O O O ...,need_help
1,BOS above problem how will it solve ?? EOS,O O O O O O O O O,need_help
2,BOS Hello sir is there anyone EOS,O O O O O O O,Greetings
3,BOS i think canvas login problem was due to br...,O O O O O O O O O O O O O O O O O O,great
4,BOS Please provide me with the link to python ...,O O O O O O O O O O O,community_class_link


In [9]:
df=df.drop('B',axis=1)

In [10]:
df['encoded_cat'] = df['category'].astype('category').cat.codes
df.head()

,text,category,encoded_cat
0,BOS Sir I created a new env with python 3.7 bu...,need_help,93
1,BOS above problem how will it solve ?? EOS,need_help,93
2,BOS Hello sir is there anyone EOS,Greetings,6
3,BOS i think canvas login problem was due to br...,great,77
4,BOS Please provide me with the link to python ...,community_class_link,55


In [11]:
df['category'].value_counts()

need_help                  781
O                          609
Greetings                  331
Thank_you                  284
good_bye                   169
                          ... 
str_method_output            1
word_file_input_query        1
zoom_mute_disable_reply      1
certificates_enquery         1
bot_timing                   1
Name: category, Length: 151, dtype: int64

In [12]:
df.tail()

,text,category,encoded_cat
3509,BOS it is not one to one resume but some advic...,need_help,93
3510,BOS hi good noon EOS,Greetings,6
3511,BOS now what will year contain EOS,code_error,52
3512,BOS I have extracted it but didn&apos;t found ...,need_help,93
3513,BOS file name or extension path name EOS,need_help,93


In [13]:
data_texts = df["text"].to_list() 
data_labels = df["encoded_cat"].to_list()

In [14]:
from sklearn.model_selection import train_test_split

# Split Train and Validation data
train_texts, val_texts, train_labels, val_labels = train_test_split(data_texts, data_labels, test_size=0.2, random_state=0)

# Keep some data for inference (testing)
train_texts, test_texts, train_labels, test_labels = train_test_split(train_texts, train_labels, test_size=0.01, random_state=0)

In [15]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

In [16]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
))

In [17]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=151)

optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss=model.compute_loss, metrics=['accuracy'])

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'vocab_layer_norm', 'activation_13', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dropout_19', 'pre_classifier', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

In [ ]:
model.fit(train_dataset.shuffle(1000).batch(16), epochs=30, batch_size=16, validation_data=val_dataset.shuffle(1000).batch(16))

Epoch 1/30
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autog

In [ ]:
save_directory = "/content" # change this to your preferred location

model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

('/content/tokenizer_config.json',
 '/content/special_tokens_map.json',
 '/content/vocab.txt',
 '/content/added_tokens.json')

In [ ]:
loaded_tokenizer = DistilBertTokenizer.from_pretrained(save_directory)
loaded_model = TFDistilBertForSequenceClassification.from_pretrained(save_directory)

Some layers from the model checkpoint at /content were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at /content and are newly initialized: ['dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
test_text='Hi! This is rudra from DLCVNLP batch.'
test_text

'Hi! This is rudra from DLCVNLP batch.'

In [ ]:
predict_input = loaded_tokenizer.encode(test_text,
                                 truncation=True,
                                 padding=True,
                                 return_tensors="tf")

output = loaded_model(predict_input)[0]

prediction_value = tf.argmax(output, axis=1).numpy()[0]
df.loc[df['encoded_cat'] == prediction_value, 'category'].iloc[0]

'Greetings'